<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=8807e801994679fa01b5fb5b041672e6fe2fc73d122160b59267e2358a9414a0
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-17 10:21:00
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.40 C
-------------------
Today PnL: -38.08 K (-0.27%)
Current PnL: -27.14 L (-17.53%)
CY Booked + Current PnL: -12.85 L (-8.3%)
-------------------
Total profit:  1.36 L
Total loss:  -28.50 L
-------------------
Total Booked + Current PnL: 14.13 L (11.06%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.21%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 91.77 L (65.59%)
Deployed:  1.28 C
Current:  1.40 C
CAGR/XIRR %: 5.02%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-0.12,-18.18,22.33,0.09,18413.0,-18319.0,82458.0,91.46,35.0,M-SC,2.47,253.0,-0.99,0.59,8.33,OX40N,NTT,DURABLES
82,VOLTAS,1530.00,-0.11,8.95,9.86,19.69,20598.0,17163.0,208905.0,-1.63,58.0,X-MC,2.56,78.0,0.83,1.50,16.36,XY25,NTT,AC
50,MASFIN,398.61,-0.49,-3.26,26.16,22.05,24796.0,-3195.0,94785.0,-16.44,51.0,H-SC,6.88,164.0,-0.13,0.68,37.78,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,-1.45,-14.66,17.20,0.02,25624.0,-25585.0,148974.0,127.12,45.0,M-SC,14.36,234.0,-1.00,1.07,29.58,OX40N,NTT,PAINTS
43,ITC,452.00,0.07,-1.47,12.44,10.78,29455.0,-3540.0,236778.0,-41.15,46.0,X-LC,1.97,5.0,-0.12,1.70,3.69,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RAJOOENG,143.1,2.47,-34.13,111.94,39.61,75582.0,-34980.0,67520.0,-55.45,39.0,H-SC,29.29,135.0,-0.46,0.48,6.38,AR,ATH,MISC
63,SFL,1287.0,1.98,-43.19,122.92,26.63,183097.0,-113262.0,148956.0,12.80,33.0,M-SC,11.84,239.0,-0.62,1.07,1.98,XY24,NTT,MISC
13,BSOFT,831.7,1.26,-21.78,92.77,50.79,102066.0,-30633.0,110020.0,0.50,65.0,H-SC,6.44,151.0,-0.30,0.79,28.10,XR,ATH,IT
16,CERA,9475.0,1.02,-29.30,75.24,23.89,93569.0,-51542.0,124361.0,-31.74,44.0,H-SC,10.76,157.0,-0.55,0.89,7.40,OX40N,NTT,CERAMICS
75,TMPV,600.0,0.70,-28.70,72.49,22.98,116749.0,-64843.0,161055.0,-25.52,33.0,X-LC,6.73,3.0,-0.56,1.16,1.30,XY24,NTT,AUTO


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COFFEEDAY,80.00,-2.26,-42.12,133.71,35.27,87877.0,-47827.0,65722.0,-55.73,44.0,L-SC,10.66,270.0,-0.54,0.47,60.86,XR,NTT,HOTELS
48,KPIGREEN,730.66,-1.86,-16.10,75.16,46.95,79010.0,-20175.0,105122.0,-22.59,41.0,H-SC,7.98,154.0,-0.26,0.75,31.69,MH,ATH,POWER
22,DIXON,18931.72,-1.78,-10.87,41.44,26.07,72108.0,-21216.0,174005.0,-56.66,38.0,X-MC,9.64,56.0,-0.29,1.25,8.37,X40N,ATH,IT
71,TANLA,1943.92,-1.77,-36.82,247.75,119.69,423786.0,-99707.0,171054.0,-36.34,43.0,H-SC,7.95,156.0,-0.24,1.23,35.13,AR,ATH,IT
19,COLPAL,3726.84,-1.72,-19.38,75.53,41.51,160365.0,-51045.0,212320.0,-3.88,40.0,X-MC,7.31,61.0,-0.32,1.52,1.01,XY25,ATH,FMCG


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.48,-1.73,111.58,107.91,161885.0,-2560.0,145084.0,-20.15,58.0,M-SC,12.52,216.0,-0.02,1.04,8.18,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.45,-14.66,17.20,0.02,25624.0,-25585.0,148974.0,127.12,45.0,M-SC,14.36,234.0,-1.00,1.07,29.58,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.12,-18.18,22.33,0.09,18413.0,-18319.0,82458.0,91.46,35.0,M-SC,2.47,253.0,-0.99,0.59,8.33,OX40N,NTT,DURABLES
66,SIS,528.00,0.02,-24.17,59.90,21.26,50436.0,-26831.0,84201.0,1986.53,50.0,H-SC,5.06,166.0,-0.53,0.60,13.76,OX40N,NTT,MISC
47,KANSAINER,340.00,-0.49,-25.44,52.19,13.47,104938.0,-68598.0,201069.0,-69.28,34.0,H-SC,5.65,158.0,-0.65,1.44,2.24,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.57,6.93,61.1,72.27,110709.0,11745.0,181193.0,-8.35,65.0,M-LC,2.6,99.0,0.11,1.3,13.53,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.57,6.93,61.10,72.27,110709.0,11745.0,181193.0,-8.35,65.0,M-LC,2.60,99.0,0.11,1.30,13.53,XR,NTT,IT
25,FINCABLES,1641.55,-0.48,-1.73,111.58,107.91,161885.0,-2560.0,145084.0,-20.15,58.0,M-SC,12.52,216.0,-0.02,1.04,8.18,OX40N,ATH,CABLES
38,INDIAMART,4810.62,0.54,-4.03,115.40,106.72,136593.0,-4971.0,118365.0,-52.54,36.0,H-SC,3.29,139.0,-0.04,0.85,18.41,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,0.37,-3.22,40.14,35.63,81413.0,-6751.0,202823.0,-16.74,38.0,H-MC,2.85,119.0,-0.08,1.46,12.91,AR,ATH,PHARMA
37,IEX,219.00,0.29,-4.44,55.53,48.63,106106.0,-8875.0,191079.0,-36.48,47.0,H-SC,14.08,136.0,-0.08,1.37,6.60,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.65,-18.78,91.42,55.47,180650.0,-45681.0,197605.0,-28.03,23.0,X-MC,13.42,64.0,-0.25,1.42,1.25,X40N,ATH,FINANCE
53,PAGEIND,51605.07,-0.58,-10.27,42.77,28.11,77296.0,-20685.0,180725.0,-34.99,26.0,X-MC,13.20,55.0,-0.27,1.30,0.00,X40,ATH,APPARELS
8,AWL,485.00,-1.09,-23.90,101.71,53.50,233557.0,-72121.0,229630.0,-63.12,28.0,X-MC,3.29,58.0,-0.31,1.65,1.99,XY24,NTT,FMCG
11,BATAINDIA,2096.00,-0.94,-39.36,122.98,35.20,95949.0,-50650.0,78020.0,2.24,29.0,X-SC,16.29,92.0,-0.53,0.56,0.00,X40,NTT,FOOTWEAR
33,HINDUNILVR,2922.00,0.00,-11.26,28.10,13.67,64099.0,-28953.0,228110.0,-17.95,32.0,X-LC,2.30,14.0,-0.45,1.64,7.69,XY25,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,950.0,-0.56,-1.78,26.73,24.48,66121.0,-4488.0,247368.0,-81.32,42.0,X-SC,0.26,86.0,-0.07,1.78,20.94,X40N,NTT,MISC
43,ITC,452.0,0.07,-1.47,12.44,10.78,29455.0,-3540.0,236778.0,-41.15,46.0,X-LC,1.97,5.0,-0.12,1.70,3.69,X40,NTT,FMCG
20,DABUR,735.0,-0.01,-2.80,47.84,43.70,115351.0,-6950.0,241118.0,-10.89,38.0,X-MC,2.29,72.0,-0.06,1.73,11.83,XY24,BTT,FMCG
33,HINDUNILVR,2922.0,0.00,-11.26,28.10,13.67,64099.0,-28953.0,228110.0,-17.95,32.0,X-LC,2.30,14.0,-0.45,1.64,7.69,XY25,NTT,FMCG
12,BERGEPAINT,680.0,-0.75,-4.51,26.87,21.14,58329.0,-10259.0,217080.0,-11.33,38.0,X-MC,2.37,74.0,-0.18,1.56,22.36,XY24,NTT,PAINTS


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.14,-25.66,120.80,64.15,213695.0,-61051.0,176900.0,-56.31,33.0,X-SC,2.78,82.0,-0.29,1.27,0.00,XY24,BTT,CEMENT
11,BATAINDIA,2096.00,-0.94,-39.36,122.98,35.20,95949.0,-50650.0,78020.0,2.24,29.0,X-SC,16.29,92.0,-0.53,0.56,0.00,X40,NTT,FOOTWEAR
53,PAGEIND,51605.07,-0.58,-10.27,42.77,28.11,77296.0,-20685.0,180725.0,-34.99,26.0,X-MC,13.20,55.0,-0.27,1.30,0.00,X40,ATH,APPARELS
31,HAVELLS,2069.16,-1.05,-10.54,48.09,32.48,141102.0,-34579.0,293412.0,-15.05,38.0,X-MC,6.11,67.0,-0.25,2.11,0.53,X40,ATH,ELECTRICAL
19,COLPAL,3726.84,-1.72,-19.38,75.53,41.51,160365.0,-51045.0,212320.0,-3.88,40.0,X-MC,7.31,61.0,-0.32,1.52,1.01,XY25,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-0.77,-29.93,103.85,42.84,47306.0,-19454.0,45552.0,-54.27,41.0,X-SC,35.36,83.0,-0.41,0.33,3.06,XY24,NTT,MISC
59,RELAXO,1176.00,0.16,-47.98,192.79,52.31,145576.0,-69650.0,75510.0,-44.63,40.0,X-SC,7.09,91.0,-0.48,0.54,1.07,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.94,-39.36,122.98,35.20,95949.0,-50650.0,78020.0,2.24,29.0,X-SC,16.29,92.0,-0.53,0.56,0.00,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,-0.09,-4.90,30.48,24.08,36796.0,-6227.0,120723.0,-12.55,32.0,X-SC,5.16,89.0,-0.17,0.87,13.75,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.74,-17.09,72.86,43.32,98390.0,-27832.0,135040.0,-28.80,35.0,X-SC,4.89,90.0,-0.28,0.97,3.69,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,0.55,-11.39,36.22,20.71,121395.0,-43076.0,335161.0,-23.62,62.0,X-LC,4.24,1.0,-0.35,2.41,11.99,X40,ATH,IT
41,INFY,1972.00,0.26,8.72,23.48,34.24,79870.0,27273.0,340161.0,-15.29,60.0,X-LC,4.26,2.0,0.34,2.44,17.72,X40,NTT,IT
75,TMPV,600.00,0.70,-28.70,72.49,22.98,116749.0,-64843.0,161055.0,-25.52,33.0,X-LC,6.73,3.0,-0.56,1.16,1.30,XY24,NTT,AUTO
81,VBL,671.64,-0.51,-4.11,41.49,35.67,125656.0,-12983.0,302859.0,-15.46,53.0,X-LC,3.99,4.0,-0.10,2.17,9.17,X40N,ATH,FMCG
43,ITC,452.00,0.07,-1.47,12.44,10.78,29455.0,-3540.0,236778.0,-41.15,46.0,X-LC,1.97,5.0,-0.12,1.70,3.69,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-0.11,-16.91,118.64,81.67,92479.0,-15861.0,77949.0,7005.56,45.0,L-SC,12.04,271.0,-0.17,0.56,50.95,XR,NTT,CERAMICS
50,MASFIN,398.61,-0.49,-3.26,26.16,22.05,24796.0,-3195.0,94785.0,-16.44,51.0,H-SC,6.88,164.0,-0.13,0.68,37.78,XR,ATH,FINANCE
13,BSOFT,831.70,1.26,-21.78,92.77,50.79,102066.0,-30633.0,110020.0,0.50,65.0,H-SC,6.44,151.0,-0.30,0.79,28.10,XR,ATH,IT
39,INDIGOPNTS,1408.00,-1.45,-14.66,17.20,0.02,25624.0,-25585.0,148974.0,127.12,45.0,M-SC,14.36,234.0,-1.00,1.07,29.58,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,0.39,-2.07,23.50,20.95,57941.0,-5210.0,246558.0,-3.02,50.0,X-LC,17.16,31.0,-0.09,1.77,32.83,X40,ATH,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,1.26,-21.78,92.77,50.79,102066.0,-30633.0,110020.0,0.50,65.0,H-SC,6.44,151.0,-0.30,0.79,28.10,XR,ATH,IT
84,WIPRO,420.00,0.57,6.93,61.10,72.27,110709.0,11745.0,181193.0,-8.35,65.0,M-LC,2.60,99.0,0.11,1.30,13.53,XR,NTT,IT
41,INFY,1972.00,0.26,8.72,23.48,34.24,79870.0,27273.0,340161.0,-15.29,60.0,X-LC,4.26,2.0,0.34,2.44,17.72,X40,NTT,IT
1,ABB,7934.00,-0.07,-3.89,51.46,45.57,129395.0,-10171.0,251448.0,-38.27,56.0,H-MC,4.55,120.0,-0.08,1.81,6.99,AR,NTT,ELECTRICAL
73,TCS,4389.97,0.55,-11.39,36.22,20.71,121395.0,-43076.0,335161.0,-23.62,62.0,X-LC,4.24,1.0,-0.35,2.41,11.99,X40,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.29
1,25,44.86
2,50,76.67


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.90
MC    30.91
LC    24.17
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.69
X40      23.03
X40N     13.04
XR        9.75
XY25      9.08
AR        9.07
OX40N     7.58
SR        0.99
MH        0.75
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.42
H-SC    23.75
X-LC    20.72
M-SC    11.69
X-SC     8.06
H-MC     4.83
L-SC     1.40
M-MC     1.32
M-LC     1.30
H-LC     1.13
L-LC     1.02
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.37,-7.87,42.56
IT,13.03,-15.86,74.11
FINANCE,9.44,-21.20,73.91
MISC,7.29,-27.85,80.15
ELECTRICAL,6.02,-12.04,53.08
PAINTS,5.84,-13.48,30.34
INSURANCE,4.77,-2.21,37.18
PHARMA,4.14,-1.97,34.06
AUTO,2.83,-32.99,79.10


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3205882.0,21
AR,1309823.0,10
XR,1302609.0,13
X40,1017666.0,14
X40N,869156.0,9
OX40N,734646.0,10
XY25,376155.0,6
SR,282126.0,2
MH,79010.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3667776.0,25
M-SC,1431326.0,15
X-MC,1425758.0,16
X-LC,842293.0,11
X-SC,798109.0,8
H-MC,406626.0,3
L-SC,265494.0,3
M-LC,110709.0,1
H-LC,75787.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1286584.0      6
           AR         896970.0      5
M-SC       XY24       821971.0      6
H-SC       XR         782294.0      7
X-MC       X40        497114.0      7
           XY24       407237.0      3
H-SC       OX40N      340792.0      4
X-MC       X40N       340444.0      4
X-LC       X40        330327.0      5
X-SC       X40N       310087.0      3
M-SC       OX40N      308716.0      5
X-SC       XY24       297797.0      3
H-SC       SR         282126.0      2
X-LC       X40N       218625.0      2
H-MC       AR         210808.0      2
X-LC       XY24       196475.0      2
H-MC       XY24       195818.0      1
X-SC       X40        190225.0      2
X-MC       XY25       180963.0      2
L-SC       XR         180356.0      2
M-SC       XR         174381.0      2
           AR         126258.0      2
M-LC       XR         110709.0      1
X-LC       XY25        96866.0      2
L-SC       OX40N       85138.0      1
H-SC       MH          79010.0      1
H-LC       AR          75787.0      1
M-MC       XY25        60894.0      1
L-MC       XR          54869.0      1
L-LC       XY25        37432.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 34.0 seconds
